In [2]:
import os
import numpy as np
import tensorflow as tf

/Users/sungchulchoi/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
IMAGE_DIR = "Images"
ANNOTATION_DIR = "Annotation"

bleeds = os.listdir(ANNOTATION_DIR)

bleeds[:5]

['n02085620-Chihuahua',
 'n02085782-Japanese_spaniel',
 'n02085936-Maltese_dog',
 'n02086079-Pekinese',
 'n02086240-Shih-Tzu']

In [4]:
image_set = {}
annotation_set = {}

In [5]:
for bleed in bleeds:
    image_dir = os.path.join(IMAGE_DIR, bleed)
    annotation_dir = os.path.join(ANNOTATION_DIR, bleed)
    image_set[bleed] = os.listdir(image_dir)
    annotation_set[bleed] = os.listdir(annotation_dir)

In [6]:
total_data = []
for bleed in image_set.keys():
    total_data.extend([ [filename,bleed] for filename in image_set[bleed]])
total_data = np.array(total_data)

In [7]:
total_data[:10]

array([['n02085620_10074.jpg', 'n02085620-Chihuahua'],
       ['n02085620_10131.jpg', 'n02085620-Chihuahua'],
       ['n02085620_10621.jpg', 'n02085620-Chihuahua'],
       ['n02085620_1073.jpg', 'n02085620-Chihuahua'],
       ['n02085620_10976.jpg', 'n02085620-Chihuahua'],
       ['n02085620_11140.jpg', 'n02085620-Chihuahua'],
       ['n02085620_11238.jpg', 'n02085620-Chihuahua'],
       ['n02085620_11258.jpg', 'n02085620-Chihuahua'],
       ['n02085620_11337.jpg', 'n02085620-Chihuahua'],
       ['n02085620_11477.jpg', 'n02085620-Chihuahua']], dtype='<U40')

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     total_data[:,0], total_data[:,1], test_size=0.2, random_state=42)

In [9]:
X_train[:10]

array(['n02098286_1989.jpg', 'n02099849_3592.jpg', 'n02096294_250.jpg',
       'n02108422_2623.jpg', 'n02094433_2006.jpg', 'n02113978_451.jpg',
       'n02087046_430.jpg', 'n02086646_637.jpg', 'n02107142_17111.jpg',
       'n02115641_7679.jpg'], dtype='<U40')

In [10]:
X_test[:10]

array(['n02096585_1571.jpg', 'n02113624_2224.jpg', 'n02093256_4090.jpg',
       'n02115641_2513.jpg', 'n02098286_1224.jpg', 'n02112706_2161.jpg',
       'n02111500_4315.jpg', 'n02110185_7936.jpg', 'n02102318_422.jpg',
       'n02098286_1591.jpg'], dtype='<U40')

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16463,), (4116,), (16463,), (4116,))

In [12]:
sess = tf.Session()

x_data = X_train
y_data = y_train
current_index = 0

record_location = "./output/images"

if os.path.exists("./output") is not True:
    os.mkdir("output")

In [ ]:
writer = None
current_index = 0


for images_filename,breed in zip(x_data,y_data):

    if current_index % 100 == 0:
        
        if writer:
            writer.close()
        record_filename = "{record_location}-{current_index}.tfrecords".format(
            record_location=record_location, current_index=current_index
        )

        writer = tf.python_io.TFRecordWriter(record_filename)
        file_full_path = os.path.join(IMAGE_DIR, breed,  images_filename)
                
        image_file = tf.read_file(file_full_path)
        try:
            image = tf.image.decode_jpeg(image_file)
        except:
            print("Error : ", images_filename)
            continue


        grayscale_image = tf.image.rgb_to_grayscale(image)
        resized_image = tf.image.resize_images(grayscale_image, [250, 151])
        
        image_bytes = sess.run(tf.cast(resized_image, tf.uint8)).tobytes()
        image_label = breed.encode("utf-8")
        
        example = tf.train.Example(features = tf.train.Features(
                                    feature={'label': 
                                              tf.train.Feature(bytes_list=tf.train.BytesList(
                                                  value=[image_label])), 
                                              "images":
                                              tf.train.Feature(bytes_list=tf.train.BytesList(
                                                  value=[image_bytes]))
                                             }
                                  ))
        writer.write(example.SerializeToString())
    current_index += 1
    writer.close()
print("Done")

In [18]:

filenmae_queue = tf.train.string_input_producer(tf.train.match_filenames_once(record_location+"*.tfrecords"))

reader = tf.TFRecordReader()
_, serialized = reader.read(filenmae_queue)

features = tf.parse_single_example(
    serialized,
    features={
        "label" : tf.FixedLenFeature([], tf.string),
        "image" : tf.FixedLenFeature([], tf.string),
    }
)

record_image = tf.decode_raw(features['image'], tf.uint8)
image = tf.reshape(record_image, [250,151,1])
label = tf.cast(features['label'], tf.string)

min_after_dequeue = 10
batch_size = 512
capacity = min_after_dequeue + 3 * batch_size

image_batch, label_batch = tf.train.shuffle_batch([image, label], batch_size=batch_size, capacity=capacity, min_after_dequeue=min_after_dequeue)

ERROR! Session/line number was not unique in database. History logging moved to new session 238


In [19]:
float_image_batch = tf.image.convert_image_dtype(image_batch, tf.float32)

conv2d_layer_one = tf.contrib.layers.convolution2d(float_image_batch, 
                                                   num_outputs=32,
                                                   activation_fn=tf.nn.relu,
                                                   kernel_size=(5,5),
                                                   stride=(1,1),
                                                   trainable=True)
pool_layer_one = tf.nn.max_pool(conv2d_layer_one, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

pool_layer_one

Instructions for updating:
Use the retry module or similar alternatives.


<tf.Tensor 'MaxPool:0' shape=(512, 125, 76, 32) dtype=float32>

In [20]:
conv2d_layer_two = tf.contrib.layers.convolution2d(pool_layer_one, 
                                                   num_outputs=64,
                                                   activation_fn=tf.nn.relu,
                                                   kernel_size=(5,5),
                                                   stride=(1,1),
                                                   trainable=True)

pool_layer_two = tf.nn.max_pool(conv2d_layer_two, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

conv2d_layer_two, pool_layer_two

(<tf.Tensor 'Conv_1/Relu:0' shape=(512, 125, 76, 64) dtype=float32>,
 <tf.Tensor 'MaxPool_1:0' shape=(512, 63, 38, 64) dtype=float32>)

In [21]:
flattened_layer_two = tf.reshape(
    pool_layer_two,
    [
        batch_size, -1
    ]
    )


In [22]:

hidden_layer_three = tf.contrib.layers.fully_connected(
    flattened_layer_two, num_outputs = batch_size
)